<a href="https://colab.research.google.com/github/Satyam-kumar-yadav/Yolov3_Object_detection/blob/main/YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf data

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14358, done.
remote: Total 14358 (delta 0), reused 0 (delta 0), pack-reused 14358
Receiving objects: 100% (14358/14358), 13.08 MiB | 7.56 MiB/s, done.
Resolving deltas: 100% (9764/9764), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
# make darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [ ]:
!nvidia-smi

Mon Nov  2 14:41:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    37W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Step 3: Download pretrained YOLOv3 weights
YOLOv3 has been trained already on the coco dataset which has 80 classes that it can predict. We will grab these pretrained weights so that we can run YOLOv3 on these pretrained classes and get detections.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Mounted at /content/gdrive


In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'Colab Notebooks'	    'CV-Shivraj Yadav.pdf'   Oops
'CV-Shivraj Yadav (1).pdf'  'My Drive'		     yolov3


In [ ]:
# this is where my zip is stored (I created a yolov3 folder where I will get my required files from)
!ls /mydrive/yolov3

backup	generate_train.py  obj.data  obj.names	obj.zip  yolov3_custom.cfg


In [ ]:
# copy the .zip file into the root directory of cloud VM
!cp /mydrive/yolov3/obj.zip ../

In [ ]:
# unzip the zip file and its contents should now be in /darknet/data/obj
!unzip ../obj.zip -d darknet/data/


Archive:  ../obj.zip
  inflating: darknet/data/obj/000305928f7b43da.jpg  
  inflating: darknet/data/obj/000305928f7b43da.txt  
  inflating: darknet/data/obj/0005451344f10814.jpg  
  inflating: darknet/data/obj/0005451344f10814.txt  
  inflating: darknet/data/obj/000731dd242627f2.jpg  
  inflating: darknet/data/obj/000731dd242627f2.txt  
  inflating: darknet/data/obj/0027b445b545ae1c.jpg  
  inflating: darknet/data/obj/0027b445b545ae1c.txt  
  inflating: darknet/data/obj/003af63a783a1615.jpg  
  inflating: darknet/data/obj/003af63a783a1615.txt  
  inflating: darknet/data/obj/00410cf982be7b7c.jpg  
  inflating: darknet/data/obj/00410cf982be7b7c.txt  
  inflating: darknet/data/obj/00459f3bf70283d3.jpg  
  inflating: darknet/data/obj/00459f3bf70283d3.txt  
  inflating: darknet/data/obj/004eb6ca27183afe.jpg  
  inflating: darknet/data/obj/004eb6ca27183afe.txt  
  inflating: darknet/data/obj/006deca414d3ca79.jpg  
  inflating: darknet/data/obj/006deca414d3ca79.txt  
  inflating: darknet/data

In [ ]:
# download cfg to google drive and change its name
!cp cfg/yolov3.cfg /mydrive/yolov3/yolov3_custom.cfg

In [ ]:
# to download to local machine (change its name to yolov3_custom.cfg once you download)
download('cfg/yolov3.cfg')

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolov3/obj.names ./dt
!cp /mydrive/yolov3/obj.data  ./dt



In [ ]:
# upload the generate_train.py script to cloud VM from local machine 
upload()

Saving generate_train.py to generate_train.py
saved file generate_train.py


In [ ]:

!python generate_train.py

In [ ]:

%cd /content/darknet

/content/darknet


In [ ]:
# verify train.txt can be seen in our darknet/data folder

!ls data/

9k.tree     eagle.jpg	 imagenet.labels.list	   openimages.names  voc.names
coco9k.map  giraffe.jpg  imagenet.shortnames.list  person.jpg
coco.names  goal.txt	 labels			   scream.jpg
dog.jpg     horses.jpg	 obj			   train.txt


In [ ]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

URL transformed to HTTPS due to an HSTS policy
--2020-11-02 15:06:46--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.52MB/s    in 1m 55s  

2020-11-02 15:08:42 (1.35 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!ls dt/obj.data

In [ ]:

%cd /content/darknet
%cd ..

/content/darknet
/content


In [ ]:
!ls dt/obj.data

dt/obj.data


In [ ]:
%cd darknet/

/content/darknet


In [ ]:
# train your custom detector
!./darknet detector train dt_1/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.451928, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 241.125656, iou_loss = 0.000000, total_loss = 241.125656 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.439168, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 894.880371, iou_loss = 0.000000, total_loss = 894.880371 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.559890, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 5711.081055, iou_loss = 0.000000, total_loss = 5711.081055 

 8: 2279.277100, 2280.050781 avg loss, 0.000000 rate, 22.772548 seconds, 512 images, 37.538980 hours left
Loaded: 0.000039 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cl

In [ ]:
imShow('chart.png')

In [ ]:
!./darknet detector train dt_1/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000062, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000021, iou_loss = 0.000000, total_loss = 0.000021 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000026, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000015, iou_loss = 0.000000, total_loss = 0.000015 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000130, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000019, iou_loss = 0.000000, total_loss = 0.000019 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000062, .5R: 0.000000, .75R: 0.000000, count: 1,

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..


/content/darknet/cfg
/content/darknet


In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test dt_1/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights /mydrive/images/gun_detect.jpg -thresh 0.3
imShow('predictions.jpg')